In [1]:
import pickle
import pandas as pd
import os           
import numpy as np
import re
import itertools
import plotly.express as px
#import brightway2 as bw 
import random 
import string

In [2]:
import sys
sys.path.append('../../Module')  #two levels up & then down to Module folder
from common_mod import *
from mod_resIC import *

<h3> BW2 CML 2001 does not have ADP (fossil) IC, only comp. 3 sources </h3>

In [3]:
LCIAname = ["CML", "SP", "openLCA"]  #source LCIA name 
lcia_name = "CML_Abiotic depletion (fossil)"            # name will be used when saving data/plots
SP_name = "Abiotic depletion (fossil)"
cml_col_name = "ADPfossil_MJ"
cml_col_keep = [2,14,15,16,17] # these are "unit",'cas_number','flow'... to keep for each IC, will append cml_col_name later
olca_filepathname = "../../Data_rawCFs/olca_CML_ADP_fossil_saved.dat"
#bw2_filepathname =  "../../Data_rawCFs/ "
#used in final calculation to create dict names
pairwise_name = ["SPvsCML", "SPvsopenLCA",  "openLCAvsCML"]

<h3> read in raw cleaned-up CML and select ADP (fossil) col </h3>

In [4]:
cml = pd.read_excel("CML_cleaned_11IC.xlsx" )  #header=[1,2,3]
cml.columns

Index(['Unnamed: 0', 'Substance', 'unit', 'ADPelement_kgSb', 'ADPfossil_MJ',
       'GWP_kgCO2', 'ODP_kgCFC-11', 'HTP_kg1,4DCB', 'FAETP_kg1,4DCB',
       'MAETP_kg1,4DCB', 'TETP_kg1,4DCB', 'POFP_kgEthylene', 'AP_kgSO2',
       'EP_kgPO4', 'cas_number', 'flow', 'subcategory', 'category'],
      dtype='object')

In [5]:
cml_col_keep.append(cml.columns.get_loc(cml_col_name) )
cml_col_keep

[2, 14, 15, 16, 17, 4]

In [6]:
cml_raw = cml.iloc[: , cml_col_keep]
print(len(cml_raw))
cml_raw = cml_raw.dropna(subset=[cml_col_name])  #drop all na value for the IC
print(len(cml_raw))
cml_raw = cml_raw.rename(columns={cml_col_name:"value"  })
cml_raw = cml_raw.rename(str.lower, axis='columns')
#cml_raw.head()

1961
9


In [7]:
cml_raw['flow'] = cml_raw['flow'].str.lower()
cml_raw

,unit,cas_number,flow,subcategory,category,value
84,kg,coal hard,coal hard,res_landscape_resources,resourcesIC,27.91
85,kg,coal soft,"coal soft, lignite",res_landscape_resources,resourcesIC,13.96
86,m3,008006-14-2,natural gas,res_landscape_resources,resourcesIC,38.84
87,kg,008012-95-1,oil crude,res_landscape_resources,resourcesIC,41.87
88,MJ,fossil- f-u,fossil fuel,res_landscape_resources,resourcesIC,1.00
89,MJ,coal hard,coal hard,res_landscape_resources,resourcesIC,1.00
90,MJ,coal soft,"coal soft, lignite",res_landscape_resources,resourcesIC,1.00
91,MJ,008006-14-2,natural gas,res_landscape_resources,resourcesIC,1.00
92,MJ,008012-95-1,oil crude,res_landscape_resources,resourcesIC,1.00


<font color = 'red'> 9 flows for raw CML, diff. naming from other LCIA sources, can't be renamed consistently  </font> 

<h3> read in raw SP </h3>

In [8]:
SP_cml = pd.read_excel("../../Data_rawCFs/SP_CML-IA.xlsx", sheet_name=SP_name, 
                                  header=[0], converters={'CAS':str})

print(len(SP_cml))
SP_cml = SP_cml.rename(columns={"Compartment": "category", "Subcompartment": "subcategory", 
                               "Substance": "flow", "Factor":"value", "CAS": "cas_number" })
SP_cml = SP_cml.rename(str.lower, axis='columns')

37


In [9]:
SP_cml['flow'] = SP_cml['flow'].str.lower()
SP_cml.head()

,category,subcategory,flow,cas_number,value,unit
0,Raw,(unspecified),"coal, 18 mj per kg",NaN,18.00,MJ / kg
1,Raw,(unspecified),"coal, 26.4 mj per kg",NaN,26.40,MJ / kg
2,Raw,(unspecified),"coal, 29.3 mj per kg",NaN,29.30,MJ / kg
3,Raw,(unspecified),"coal, bituminous, 24.8 mj per kg",NaN,24.80,MJ / kg
4,Raw,(unspecified),"coal, brown",NaN,8.75,MJ / kg


In [10]:
for x in SP_cml['flow']: print(x)

coal, 18 mj per kg
coal, 26.4 mj per kg
coal, 29.3 mj per kg
coal, bituminous, 24.8 mj per kg
coal, brown
coal, brown, 10 mj per kg
coal, brown, 8 mj per kg
coal, feedstock, 26.4 mj per kg
coal, hard
coal, hard, 30.7 mj per kg
energy, from coal
energy, from coal, brown
energy, from gas, natural
energy, from oil
energy, from sulfur
energy, unspecified
gas, mine, off-gas, process, coal mining/kg
gas, mine, off-gas, process, coal mining/m3
gas, natural, 30.3 mj per kg
gas, natural, 35 mj per m3
gas, natural, 36.6 mj per m3
gas, natural, 46.8 mj per kg
gas, natural, feedstock, 35 mj per m3
gas, natural, feedstock, 46.8 mj per kg
gas, natural/kg
gas, natural/m3
gas, off-gas, oil production
gas, petroleum, 35 mj per m3
methane
oil, crude
oil, crude, 38400 mj per m3
oil, crude, 41 mj per kg
oil, crude, 42 mj per kg
oil, crude, 42.6 mj per kg
oil, crude, 42.7 mj per kg
oil, crude, feedstock, 41 mj per kg
oil, crude, feedstock, 42 mj per kg


<h3> read in raw openLCA dat </h3>

In [11]:
olca_cml = unpickle_data(olca_filepathname)
olca_cml = olca_cml.rename(str.lower, axis='columns')
olca_cml['flow'] = olca_cml['flow'].str.lower()
olca_cml.head()

,impact_method,impact_category_uuid,impact_category,reference_unit,flow_uuid,flow,cas_number,subcategory,category,unit,value
0,CML-IA baseline,a30f30eb-d5ff-3051-80f2-2f789db52886,Abiotic depletion (fossil fuels),MJ,ed5c032f-fb99-3e66-a590-0c007326efa5,"gas, off-gas, oil production",NaN,in ground,Resource,m3,35.85
1,CML-IA baseline,a30f30eb-d5ff-3051-80f2-2f789db52886,Abiotic depletion (fossil fuels),MJ,7ae371b0-8532-11e0-9d78-0800200c9a66,"coal, hard, 30.7 mj per kg, in ground",NaN,in ground,Resource,kg,30.7
2,CML-IA baseline,a30f30eb-d5ff-3051-80f2-2f789db52886,Abiotic depletion (fossil fuels),MJ,71e1cd1c-c0f1-35dd-b0d8-fbc23850d056,"coal, brown, 8 mj per kg, in ground",NaN,in ground,Resource,kg,8
3,CML-IA baseline,a30f30eb-d5ff-3051-80f2-2f789db52886,Abiotic depletion (fossil fuels),MJ,5df3c6a9-a888-4b5d-9a94-f3f610d38756,"gas, natural, feedstock, 35 mj per m3",008006-14-2,in ground,Resource,m3,35
4,CML-IA baseline,a30f30eb-d5ff-3051-80f2-2f789db52886,Abiotic depletion (fossil fuels),MJ,3ed5f377-344f-423a-b5ec-9a9a1162b944,"gas, mine, off-gas, process, coal mining",NaN,in ground,Resource,m3,35.85


<h3> unlike emission-type IC, for resources, no "common_category" will be added </h3>

<h4> <font color = 'red'> for the same one flow, it typically has same CF value regardless of its compartment (except for fresh water/seawater) </font> </h4>

using res_check_EF_value (df, df_EF_name_col, df_EF_value_col, EF_list):  #df_EF_col_name is "flow"   

### final calculation, pair-wise comparison, corr. matrix

<h4> <font color = 'red'> for raw CML, it uses diff. naming from all other sources, no same flow name found for CML vs. other LCIA sources </font> </h4>


In [12]:
# create 6 empty dict: dict_SPvsRIVM; dict_SPvsBW2; dict_SPvsopenLCA; dict_openLCAvsBW2; dict_openLCAvsRIVM; dict_BW2vsRIVM
# each dict with four keys and null values (to be assigned later)
for name in pairwise_name:
    #print(name)
    keys = [name+"_commonEF_sumtable", name+"_commonEF_list", name + "_result_diffEFs", name + "_result_corr" ]
    globals()['dict_%s' % name] = dict.fromkeys(keys, None)

In [13]:
k_list = list(dict_SPvsCML)
dict_SPvsCML[k_list[0]], dict_SPvsCML[k_list[1]] = res_compare_included_EF(SP_cml, "flow", cml_raw,  "flow", to_print = "yes" )
if len(dict_SPvsCML[k_list[1]] ) == 0: 
    dict_SPvsCML[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'CML_value': np.NaN }, index=[0])
else:
    dict_SPvsCML[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_cml, "flow", "value", cml_raw, "flow", "value", dict_SPvsCML[k_list[1]])[0] , 
         'CML_value': final_EF_combined(SP_cml, "flow", "value", cml_raw, "flow", "value", dict_SPvsCML[k_list[1]])[1] }, 
        index = dict_SPvsCML[k_list[1]]
    )

EF in A but not in B: ['oil, crude', 'coal, 18 mj per kg', 'energy, from sulfur', 'gas, natural, feedstock, 46.8 mj per kg', 'energy, from coal', 'coal, brown, 8 mj per kg', 'gas, natural, feedstock, 35 mj per m3', 'gas, mine, off-gas, process, coal mining/m3', 'oil, crude, 38400 mj per m3', 'oil, crude, 42 mj per kg', 'gas, natural/kg', 'energy, from gas, natural', 'coal, brown, 10 mj per kg', 'coal, hard, 30.7 mj per kg', 'gas, natural, 30.3 mj per kg', 'coal, brown', 'coal, bituminous, 24.8 mj per kg', 'oil, crude, feedstock, 41 mj per kg', 'energy, unspecified', 'coal, hard', 'gas, natural, 36.6 mj per m3', 'gas, natural/m3', 'gas, off-gas, oil production', 'gas, petroleum, 35 mj per m3', 'coal, feedstock, 26.4 mj per kg', 'gas, mine, off-gas, process, coal mining/kg', 'coal, 29.3 mj per kg', 'oil, crude, 42.7 mj per kg', 'energy, from coal, brown', 'coal, 26.4 mj per kg', 'gas, natural, 46.8 mj per kg', 'oil, crude, 41 mj per kg', 'oil, crude, 42.6 mj per kg', 'energy, from oil', 

In [14]:
k_list = list(dict_SPvsopenLCA)
dict_SPvsopenLCA[k_list[0]],dict_SPvsopenLCA[k_list[1]] = res_compare_included_EF(SP_cml, "flow", olca_cml,  "flow", to_print = "yes" )
if len(dict_SPvsopenLCA[k_list[1]] ) == 0: 
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame ({'SP_value': np.NaN , 'olca_value': np.NaN }, index=[0])
else:
    dict_SPvsopenLCA[k_list[2]] = pd.DataFrame (
        {'SP_value': final_EF_combined(SP_cml, "flow", "value", olca_cml, "flow", "value", dict_SPvsopenLCA[k_list[1]])[0] , 
         'olca_value': final_EF_combined(SP_cml, "flow", "value", olca_cml, "flow", "value", dict_SPvsopenLCA[k_list[1]])[1] },
        index = dict_SPvsopenLCA[k_list[1]]
    )

EF in A but not in B: []
EF in B but not in A: ['oil, crude, feedstock, 41 mj per kg, in ground', 'coal, 18 mj per kg, in ground', 'oil, crude, in ground', 'coal, hard, unspecified, in ground', 'oil, crude, feedstock, 42 mj per kg, in ground', 'coal, brown, 8 mj per kg, in ground', 'oil, crude, 41 mj per kg, in ground', 'coal, feedstock, 26.4 mj per kg, in ground', 'gas, mine, off-gas, process, coal mining', 'oil, crude, 42.6 mj per kg, in ground', 'gas, natural, in ground', 'coal, brown, in ground', 'oil, crude, 38400 mj per m3, in ground', 'gas, petroleum, 35 mj per m3, in ground', 'coal, brown, 10 mj per kg, in ground', 'gas, off-gas, oil production, in ground', 'coal, hard, 30.7 mj per kg, in ground', 'coal, hard, unspecified']
N of common EFs to be used is: 37


In [15]:
k_list = list(dict_openLCAvsCML)
dict_openLCAvsCML[k_list[0]],dict_openLCAvsCML[k_list[1]] = res_compare_included_EF(olca_cml, "flow", cml_raw, "flow", to_print = "no" )

if len(dict_openLCAvsCML[k_list[1]] ) == 0: 
    dict_openLCAvsCML[k_list[2]] = pd.DataFrame ({'olca_value': np.NaN , 'CML_value': np.NaN }, index=[0])
else:
    dict_openLCAvsCML[k_list[2]] = pd.DataFrame (
        {'olca_value': final_EF_combined(olca_cml, "flow", "value", cml_raw, "flow", "value", 
                                       dict_openLCAvsCML[k_list[1]])[0] , 
         'CML_value': final_EF_combined(olca_cml, "flow", "value", cml_raw, "flow", "value",
                                         dict_openLCAvsCML[k_list[1]])[1] },
        index = dict_openLCAvsCML[k_list[1]]
    )

Detailed differences won't be printed out, to print, add input argument: to_print = 'yes'.  
N of common EFs to be used is: 0


<h4> Assign values to final key for each dict: result_corr_coeff  </h4>


In [16]:
for name in pairwise_name:
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    try:
        new_df_corr = new_df.corr()
    except: 
        error
    globals()['dict_%s' % name][name + "_result_corr"] = new_df_corr

In [17]:
# saving each dict_ to a single worksheet, different rows , # saving to sub-folder "results/Resources_IC/"

if not os.path.exists("results/Resources_IC"):
    os.makedirs("results/Resources_IC")

filename = lcia_name + "_pairwise_comp_result.xlsx"
writer = pd.ExcelWriter("results/Resources_IC/" + filename, engine='xlsxwriter')

for name in pairwise_name:
    sum_table = globals()['dict_%s' % name][name + "_commonEF_sumtable"]
    new_df = globals()['dict_%s' % name][name + "_result_diffEFs"]
    new_df_corr = globals()['dict_%s' % name][name + "_result_corr"]

    sum_table.to_excel(writer,sheet_name=name,startrow=0 , startcol=0)   # write first  (3rows)
    new_df_corr.to_excel(writer,sheet_name=name,startrow=5 , startcol=0) # write next   (3rows)
    new_df.to_excel(writer,sheet_name=name,startrow=10 , startcol=0)     # finally, write the resulting EFs 
     
writer.save()

In [18]:
for name in pairwise_name:
    name1, name2 = name.split("vs",1)[0], name.split("vs",1)[1]
    diff_EF = globals()['dict_%s' % name][name + "_result_diffEFs"]
    print(diff_EF)

   SP_value  CML_value
0       NaN        NaN
                                             SP_value  olca_value
oil, crude                                      43.20       43.20
coal, 18 mj per kg                              18.00       18.00
energy, from sulfur                              1.00        1.00
gas, natural, feedstock, 46.8 mj per kg         46.80       46.80
energy, from coal                                1.00        1.00
coal, brown, 8 mj per kg                         8.00        8.00
gas, natural, feedstock, 35 mj per m3           35.00       35.00
gas, mine, off-gas, process, coal mining/m3     35.85       35.85
oil, crude, 38400 mj per m3                  38400.00    38400.00
oil, crude, 42 mj per kg                        42.00       42.00
gas, natural/kg                                 43.15       43.15
energy, from gas, natural                        1.00        1.00
coal, brown, 10 mj per kg                       10.00       10.00
coal, hard, 30.7 mj per kg    